#### 1. Naver 뉴스 크롤링
- 미세먼지와 관련된 뉴스기사를 크롤링하여, 기사 본문을 training data로 사용
- Naver API 사용
- 시도 이후, 성능이 좋지 못하면 어머니들 커뮤니티에서 '미세먼지' 키워드로 크롤링할 것

In [1]:
import urllib.request
import urllib.parse

from bs4 import BeautifulSoup
def get_news_link(search_word, start, display) : 
    URL = 'https://openapi.naver.com/v1/search/news.xml?'
    # API에서 요청하는 모든 변수 URL로 전달
    sort = 'sort=sim'
    start = '&start=' + str(start) # 1 ~ 1000
    display = '&display=' + str(display) # 10 ~ 100
    query = '&query=' +  urllib.parse.quote_plus(str(search_word)) # 검색어를 입력받아 UTF-8로 전환
    URL = URL + sort + start + display + query
    headers = {
        'Host' : 'openapi.naver.com',
        'User-Agent' : 'curl/7.43.0',
        'Accept' : '*/*',
        'Content-Type' : 'application/xml',
        'X-Naver-Client-Id' : '_hHysY2yqykbR2ogbcla',
        'X-Naver-Client-Secret' : 'yNDrUnypZL'
     }
     # request로 헤더 정보 전달
    req = urllib.request.Request(URL, headers=headers)
    # header정보를 포함한 request 객체를 HTTP로 전달
    f = urllib.request.urlopen(req)
    resultXML = f.read()
    xmlsoup = BeautifulSoup(resultXML, 'html.parser')
    texts = xmlsoup.select("description")
    ls = []
    for text in texts : 
        ls.append(text.text)
    return ls[1 : ]

In [38]:
ls = get_news_link("미세먼지", 1, 100)
ls

['세계보건기구(WHO)는 <b>미세먼지</b>(PM2.5)를 발암물질 1군으로 분류하고 있다. 작은 크기의 입자인 탓에 폐포 깊숙이 침투해 호흡기 질환을 유발할 수 있어 위해성이 더 크다. 이 같은 <b>미세먼지</b>는 실외 활동만 조심하면 될까?... ',
 '<b>미세먼지</b> 원인 [연합뉴스TV 제공] 환경부, 개정안 입법 예고…질소산화물 연 16만t 삭감 전망  환경부는 사업장에서 배출하는 질소산화물(NOx)에 부과금을 도입하는 &apos;대기환경보전법 시행령·시행규칙 일부 개정안&apos;을... ',
 '환경부는 지난해 발표한 &apos;<b>미세먼지</b> 관리 종합대책&apos;의 후속 조치로 사업장 배출 오염물질에 부과하는 부담금을 질소산화물까지 확대하는 법률 개정안을 내일부터 입법예고한다고 밝혔습니다. 질소산화물... ',
 '[앵커] 앞으로 <b>미세먼지</b>와 오존을 발생시키는 질소산화물을 배출하는 사업장에 부과금이 매겨집니다. 환경부는 관련 개정안을 내일부터 한 달여 간 입법예고합니다. 최서희 기자가 보도합니다. [리포트] 공장에서... ',
 '삼겹살이 <b>미세먼지</b>를 제거한다? &lt;사진=MBC &apos;기분좋은 날&apos; 방송 캡쳐&gt; 26일, MBC &apos;기분좋은 날&apos;에서는 &apos;<b>미세먼지</b>에 관한 오해와 진실&apos;로 삼겹살을 먹으면 몸 속 <b>미세먼지</b>가 제거되는지 그 사실이 공개됐다. 사실은 그렇지... ',
 '◆…<b>미세먼지</b> 농도 &lt;사진: 연합뉴스TV&gt; 3일은 대기 확산이 원활해 대체로 청정한 날씨가 전망된다. 이날 기상청은 전국이 중국 중부 지방에 위치한 고기압의 가장자리에 들어 구름이 많은 가운데 찬 공기가 유입돼... ',
 '아시아투데이 조상은 기자(세종)=<b>미세먼지</b> 원인 중 하나로 지목되는 질소산화물(NOx) 대기배출부과금 제도가 도입된다.환경부는 3일 사업장에서 배출되는 ‘질소산화물(NOx)’에 대해 부과금을 도입하는

In [3]:
len(ls)

100

In [24]:
from itertools import chain

In [47]:
#2차원 array -> 1차원 array
sentences = "".join([i for i in list(chain.from_iterable(ls))])
sentences

'세계보건기구(WHO)는 <b>미세먼지</b>(PM2.5)를 발암물질 1군으로 분류하고 있다. 작은 크기의 입자인 탓에 폐포 깊숙이 침투해 호흡기 질환을 유발할 수 있어 위해성이 더 크다. 이 같은 <b>미세먼지</b>는 실외 활동만 조심하면 될까?... <b>미세먼지</b> 원인 [연합뉴스TV 제공] 환경부, 개정안 입법 예고…질소산화물 연 16만t 삭감 전망  환경부는 사업장에서 배출하는 질소산화물(NOx)에 부과금을 도입하는 &apos;대기환경보전법 시행령·시행규칙 일부 개정안&apos;을... 환경부는 지난해 발표한 &apos;<b>미세먼지</b> 관리 종합대책&apos;의 후속 조치로 사업장 배출 오염물질에 부과하는 부담금을 질소산화물까지 확대하는 법률 개정안을 내일부터 입법예고한다고 밝혔습니다. 질소산화물... [앵커] 앞으로 <b>미세먼지</b>와 오존을 발생시키는 질소산화물을 배출하는 사업장에 부과금이 매겨집니다. 환경부는 관련 개정안을 내일부터 한 달여 간 입법예고합니다. 최서희 기자가 보도합니다. [리포트] 공장에서... 삼겹살이 <b>미세먼지</b>를 제거한다? &lt;사진=MBC &apos;기분좋은 날&apos; 방송 캡쳐&gt; 26일, MBC &apos;기분좋은 날&apos;에서는 &apos;<b>미세먼지</b>에 관한 오해와 진실&apos;로 삼겹살을 먹으면 몸 속 <b>미세먼지</b>가 제거되는지 그 사실이 공개됐다. 사실은 그렇지... ◆…<b>미세먼지</b> 농도 &lt;사진: 연합뉴스TV&gt; 3일은 대기 확산이 원활해 대체로 청정한 날씨가 전망된다. 이날 기상청은 전국이 중국 중부 지방에 위치한 고기압의 가장자리에 들어 구름이 많은 가운데 찬 공기가 유입돼... 아시아투데이 조상은 기자(세종)=<b>미세먼지</b> 원인 중 하나로 지목되는 질소산화물(NOx) 대기배출부과금 제도가 도입된다.환경부는 3일 사업장에서 배출되는 ‘질소산화물(NOx)’에 대해 부과금을 도입하는... <b>미세먼지</b> 기승을 타고, 관련 제품을 

In [55]:
delete = "<b> </b> &apos; \xa0".split()

In [56]:
for i in delete : 
    sentences = sentences.replace( i , "")


In [57]:
sentences

'세계보건기구(WHO)는 미세먼지(PM2.5)를 발암물질 1군으로 분류하고 있다. 작은 크기의 입자인 탓에 폐포 깊숙이 침투해 호흡기 질환을 유발할 수 있어 위해성이 더 크다. 이 같은 미세먼지는 실외 활동만 조심하면 될까?... 미세먼지 원인 [연합뉴스TV 제공] 환경부, 개정안 입법 예고…질소산화물 연 16만t 삭감 전망  환경부는 사업장에서 배출하는 질소산화물(NOx)에 부과금을 도입하는 대기환경보전법 시행령·시행규칙 일부 개정안을... 환경부는 지난해 발표한 미세먼지 관리 종합대책의 후속 조치로 사업장 배출 오염물질에 부과하는 부담금을 질소산화물까지 확대하는 법률 개정안을 내일부터 입법예고한다고 밝혔습니다. 질소산화물... [앵커] 앞으로 미세먼지와 오존을 발생시키는 질소산화물을 배출하는 사업장에 부과금이 매겨집니다. 환경부는 관련 개정안을 내일부터 한 달여 간 입법예고합니다. 최서희 기자가 보도합니다. [리포트] 공장에서... 삼겹살이 미세먼지를 제거한다? &lt;사진=MBC 기분좋은 날 방송 캡쳐&gt; 26일, MBC 기분좋은 날에서는 미세먼지에 관한 오해와 진실로 삼겹살을 먹으면 몸 속 미세먼지가 제거되는지 그 사실이 공개됐다. 사실은 그렇지... ◆…미세먼지 농도 &lt;사진: 연합뉴스TV&gt; 3일은 대기 확산이 원활해 대체로 청정한 날씨가 전망된다. 이날 기상청은 전국이 중국 중부 지방에 위치한 고기압의 가장자리에 들어 구름이 많은 가운데 찬 공기가 유입돼... 아시아투데이 조상은 기자(세종)=미세먼지 원인 중 하나로 지목되는 질소산화물(NOx) 대기배출부과금 제도가 도입된다.환경부는 3일 사업장에서 배출되는 ‘질소산화물(NOx)’에 대해 부과금을 도입하는... 미세먼지 기승을 타고, 관련 제품을 가진 제약사의 홍보활동이 확산되고 있다. \xa0 코세척기 수입 유통-제약사 에스에이치팜은 최근 서울광화문광장에서 열린 조선일보 주최 ‘2018서울하프마라톤대회’에 참여, 코세척... [서울신문] 부산시교육청은 학교 미세먼지 피해 예방을 하고자 

---

### 목적 : 대화형 챗봇 구현. 
1. 사용자의 말을 인식
2. 사용자가 미세먼지 정보를 원하는지, 잡담을 원하는지에 대한 인식
3. 사용자가 미세먼지 정보를 원할 경우
> 3-1 지역 정보를 정확히 인식, 인식 못할 경우 다시 물어보아 오타 혹은 지역명에 대한 확인 요구
4. 사용자가 잡담을 원하는 경우
> 4-1. 이후 기능 구현

#### 가설 1. word2vec의 CBOW
- 2번 단계, 사용자가 정보를 원하는지 혹은 잡담을 원하는지에 대한 구분

In [12]:
from soynlp.word import WordExtractor

In [16]:
word_extractor = WordExtractor(min_count = 10, 
                              min_cohesion_forward = 0.05,
                              min_right_branching_entropy = 0.00
                            )

In [17]:
word_extractor.train(ls)

training was done. used memory 0.120 Gb


In [18]:
words = word_extractor.extract()

all cohesion probabilities was computed. # words = 110
all branching entropies was computed # words = 1330
all accessor variety was computed # words = 1330


In [21]:
from soynlp.tokenizer import LTokenizer

In [58]:
tokenizer = LTokenizer()

In [59]:
print(tokenizer.tokenize(sentences, flatten = True))

['세계보건기구(WHO)는', '미세먼지(PM2.5)를', '발암물질', '1군으로', '분류하고', '있다.', '작은', '크기의', '입자인', '탓에', '폐포', '깊숙이', '침투해', '호흡기', '질환을', '유발할', '수', '있어', '위해성이', '더', '크다.', '이', '같은', '미세먼지는', '실외', '활동만', '조심하면', '될까?...', '미세먼지', '원인', '[연합뉴스TV', '제공]', '환경부,', '개정안', '입법', '예고…질소산화물', '연', '16만t', '삭감', '전망', '환경부는', '사업장에서', '배출하는', '질소산화물(NOx)에', '부과금을', '도입하는', '대기환경보전법', '시행령·시행규칙', '일부', '개정안을...', '환경부는', '지난해', '발표한', '미세먼지', '관리', '종합대책의', '후속', '조치로', '사업장', '배출', '오염물질에', '부과하는', '부담금을', '질소산화물까지', '확대하는', '법률', '개정안을', '내일부터', '입법예고한다고', '밝혔습니다.', '질소산화물...', '[앵커]', '앞으로', '미세먼지와', '오존을', '발생시키는', '질소산화물을', '배출하는', '사업장에', '부과금이', '매겨집니다.', '환경부는', '관련', '개정안을', '내일부터', '한', '달여', '간', '입법예고합니다.', '최서희', '기자가', '보도합니다.', '[리포트]', '공장에서...', '삼겹살이', '미세먼지를', '제거한다?', '&lt;사진=MBC', '기분좋은', '날', '방송', '캡쳐&gt;', '26일,', 'MBC', '기분좋은', '날에서는', '미세먼지에', '관한', '오해와', '진실로', '삼겹살을', '먹으면', '몸', '속', '미세먼지가', '제거되는지', '그', '사실이', '공개됐다.', '사실은', '그렇지...', '◆…미세먼지', '농도', '&lt;사진:', '연합뉴스TV&

In [5]:
from gensim.models.word2vec import Word2Vec

In [66]:
model = Word2Vec(sentences)

In [69]:
model.most_similar("")

C:\Users\ledes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word '신호등' not in vocabulary"